In [1]:
#I am setting this environment to practice and validate dataset and build something out of it, hopefully!!
#and for easier analaysis, ofcourse, VSCode are not Data Engineers friendly, HAHAHA!

In [74]:
import pandas as pd
import os
import datetime


In [90]:
#bulk Df is basically block deals happend by the client.
#These clients can be FII and DII. We will build something to identfy that later on.
#for now, I am trying to consolidate the client regime, how much of Indian stock they bought.

bulk_df=pd.read_csv('../staging/bulk.csv')

#Schema changes: I am changing the schema as it is not good way to keep spaces in between.
#let's create a standard of camel casing i.e Quantity Traded -> QuantityTraded.
#Also, Let's Fix the Date Standards. 25-Aug-2023 -> 2023-08-25

new_schema={'Client Name':'ClientName','Quantity Traded':'QuantityTraded', 'Trade Price / Wght. Avg. Price':'MeanWghtPrice', 'Buy/Sell':'Transaction'}
bulk_df_cleaned=bulk_df.rename(columns=new_schema).drop(columns=['Security Name','Remarks']).drop_duplicates()
bulk_df_cleaned['Date']=pd.to_datetime(bulk_df_cleaned.Date).dt.strftime('%Y-%m-%d')
bulk_df_cleaned.head()

,Date,Symbol,ClientName,Transaction,QuantityTraded,MeanWghtPrice
0,2023-08-25,ALPA,GRAVITON RESEARCH CAPITAL LLP,BUY,133673,81.39
1,2023-08-25,AMBER,GOVERNMENT OF SINGAPORE,BUY,725239,2800.00
2,2023-08-25,ARIES,GRAVITON RESEARCH CAPITAL LLP,BUY,89917,193.17
3,2023-08-25,ASTRAMICRO,GRAVITON RESEARCH CAPITAL LLP,BUY,487675,386.36
4,2023-08-25,ATULAUTO,GRAVITON RESEARCH CAPITAL LLP,BUY,390565,457.74


In [92]:
#now that we have some cleaning done, let check the regime of Client.
#Regime of Client: Varied Stocks that single client has bought.
#PS: you will see so many reassignments, coz I hate using more variables.

ClientPortfolio=bulk_df_cleaned
tdf=ClientPortfolio.groupby(['ClientName','Date'])[['Symbol']].count()
tdf=tdf.rename(columns={'Symbol':'NoOfStocksBought'}).reset_index()
tdf=tdf.sort_values(by=['NoOfStocksBought'],ascending=False)
tdf.to_csv("../data/ClientPortFolio.csv")

tdf.head()
#I know this will only give us the Client who bought most number of stocks in a day, which it bought that is still with bulk_df_cleaned
#I have alogorithm to store all the stocks bought in fixed lenght, will implement it later on. 
# Just for guess, it has to do with bit map. HAHA!

,ClientName,Date,NoOfStocksBought
9,GRAVITON RESEARCH CAPITAL LLP,2023-08-25,28
13,HRTI PRIVATE LIMITED,2023-08-25,8
1,AAKRAYA RESEARCH LLP,2023-08-25,4
32,QE SECURITIES LLP,2023-08-25,4
22,MAHADEV MANUBHAI MAKVANA,2023-08-25,3
